In [136]:
import re
import string
import numpy as np
import pydot
import random
import random 
from unicodedata import normalize
from pickle import dump
from keras.models import load_model
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint

In [90]:
def load_doc(filename):
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [91]:
# split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split(',') for line in  lines]
    return pairs

def clean_pairs(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return np.asarray(cleaned)

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
    dump(sentences, open(filename, 'wb'))
    print('Saved: %s' % filename)

In [165]:
filename = 'Khasi_English.txt'
doc = load_doc(filename)

pairs = to_pairs(doc)
cleaned_pairs = clean_pairs(pairs)

In [166]:
len(pairs)

975

In [167]:
def save_clean_data(sentences, filename):
    dump(sentences, open(filename, 'wb'))
    print('Saved: '+filename)

In [168]:
#save_clean_data(cleaned_pairs, 'khasi-english.pkl')
for each in cleaned_pairs:
    print('{'+each[0]+'} => {'+each[1]+'}')

{khublei} => {thank you}
{balei} => {why}
{pynjaijai} => {cool off}
{rieh} => {take cover}
{peit} => {watch}
{mih} => {leave}
{phet} => {scram}
{iehnoh} => {leave}
{tan} => {pull}
{pyneh} => {pull yourself together}
{adi} => {ouch}
{ada} => {ouch}
{bam} => {eat}
{ieng} => {stand up}
{shuh} => {stand aside}
{tam} => {pick it up}
{khar} => {pick it up}
{hooid} => {yes}
{haoid} => {yes}
{em} => {no}
{nai} => {no}
{map} => {im sorry}
{kloi} => {hurry up}
{wut} => {hurry up}
{lait} => {wrong}
{kaei} => {what}
{uei} => {what}
{iei} => {what}
{ani} => {wow}
{iap} => {dead}
{hatshi} => {achoo}
{shisha} => {really}
{kumno} => {hello}
{ei} => {hello}
{biang} => {perfect}
{sngewbha} => {please}
{bew} => {fuck}
{biw} => {fuck}
{bow} => {fuck}
{iarap} => {help}
{sangeh} => {stop}
{thngan} => {hungry}
{bieij} => {crazy}
{eit} => {shit}
{bleh} => {blah}
{thik} => {precisely}
{bymlahlong} => {thats impossible}
{kamkai} => {nonsense}
{iap} => {die}
{ho} => {goodbye}
{mo} => {goodbye}
{ei} => {hi}
{kade

{kumno ngan pyniapbieij ia u tom} => {how am i going to impress tom}
{jied ia iba phi sngewtynnad tam} => {pick the one you like best}
{jied ia iba phi best tam} => {pick the one you like best}
{u tom un pynmynsaw ia phi} => {tom is going to hurt you}
{u tom un pynmynsaw ia pha} => {tom is going to hurt you}
{u tom un pynmong ia me} => {tom is going to hurt you}
{u tom un pynmong ia phi} => {tom is going to hurt you}
{u tom un pynmynsaw ia me} => {tom is going to hurt you}
{u tom un pynmong ia pha} => {tom is going to hurt you}
{ngan sa iakynduh iaphi ha ing} => {ill see you at home}
{ngan sa iakynduh iame ha ing} => {ill see you at home}
{one i dei i tiket kot} => {this is a paper ticket}
{ngi trei na bynta u tom} => {we work for tom}
{u tom u sdang ban khih} => {tom began to move}
{phi tip katno ngi lah iohnong} => {do you know how much profit we made}
{nga lah leh katba nga leh} => {i gave it my best shot}
{ngan leit sha office u tom} => {im going to toms office}
{u tom u jop iaka e

In [169]:
from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle

# load a clean dataset
def load_clean_sentences(filename):
    return load(open(filename, 'rb'))

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
    dump(sentences, open(filename, 'wb'))
    print('Saved: %s' % filename)

# load dataset
raw_dataset = load_clean_sentences('khasi-english.pkl')

# reduce dataset size
n_sentences = len(raw_dataset)
dataset = raw_dataset[:n_sentences, :]
# random shuffle
shuffle(dataset)
# split into train/test
train, test = dataset[:], dataset[900:]
# save
save_clean_data(dataset, 'khasi-english.pkl')
save_clean_data(train, 'khasi-english-train.pkl')
save_clean_data(test, 'khasi-english-test.pkl')

Saved: khasi-english.pkl
Saved: khasi-english-train.pkl
Saved: khasi-english-test.pkl


In [170]:
from keras.preprocessing.text import Tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

def max_length(lines):
    return max(len(line.split()) for line in lines)

In [171]:
kha_tokenizer = create_tokenizer(cleaned_pairs[:, 0])
print(kha_tokenizer)
kha_vocab_size = len(kha_tokenizer.word_index) + 1
kha_length = max_length(cleaned_pairs[:, 0])
print('Khasi Vocabulary Size: %d' % kha_vocab_size)
print('Khasi Max Length: %d' % (kha_length))
# prepare german tokenizer
eng_tokenizer = create_tokenizer(cleaned_pairs[:, 1])
print(eng_tokenizer)
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(cleaned_pairs[:, 1])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))

Khasi Vocabulary Size: 784
Khasi Max Length: 9
English Vocabulary Size: 827
English Max Length: 11


In [172]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
def encode_sequences(tokenizer, length, lines):
    # integer encode sequences
    X = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    X = pad_sequences(X, maxlen=length, padding='post')
    return X

# one hot encode target sequence
def encode_output(sequences, vocab_size):
    ylist = list()
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
    y = array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y



In [173]:
# prepare training data
trainX = encode_sequences(eng_tokenizer, eng_length, cleaned_pairs[:, 1])
trainY = encode_sequences(kha_tokenizer, kha_length, cleaned_pairs[:, 0])
trainY = encode_output(trainY, kha_vocab_size)
# prepare validation data
testX = encode_sequences(eng_tokenizer, eng_length, test[:, 1])
testY = encode_sequences(kha_tokenizer, kha_length, test[:, 0])
testY = encode_output(testY, kha_vocab_size)

In [174]:
# define NMT model
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
    model = Sequential()
    model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
    model.add(LSTM(n_units))
    model.add(RepeatVector(tar_timesteps))
    model.add(LSTM(n_units, return_sequences=True))
    
    model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
    return model

# define model
model = define_model(eng_vocab_size, kha_vocab_size, eng_length, kha_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')
# summarize defined model
print(model.summary())
#plot_model(model, to_file='model.png', show_shapes=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 11, 256)           211712    
_________________________________________________________________
lstm_13 (LSTM)               (None, 256)               525312    
_________________________________________________________________
repeat_vector_7 (RepeatVecto (None, 9, 256)            0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 9, 256)            525312    
_________________________________________________________________
time_distributed_7 (TimeDist (None, 9, 784)            201488    
Total params: 1,463,824
Trainable params: 1,463,824
Non-trainable params: 0
_________________________________________________________________
None


In [156]:
print(trainY.shape)
print(trainX.shape)

(975, 9, 784)
(975, 11)


In [157]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=60, batch_size=64, validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)

Train on 975 samples, validate on 75 samples
Epoch 1/60
 - 73s - loss: 5.9633 - val_loss: 4.2160

Epoch 00001: val_loss improved from inf to 4.21602, saving model to model.h5
Epoch 2/60
 - 21s - loss: 3.5579 - val_loss: 3.3959

Epoch 00002: val_loss improved from 4.21602 to 3.39592, saving model to model.h5
Epoch 3/60
 - 21s - loss: 3.2088 - val_loss: 3.2260

Epoch 00003: val_loss improved from 3.39592 to 3.22604, saving model to model.h5
Epoch 4/60
 - 21s - loss: 3.0577 - val_loss: 3.0938

Epoch 00004: val_loss improved from 3.22604 to 3.09380, saving model to model.h5
Epoch 5/60
 - 21s - loss: 2.9324 - val_loss: 2.9517

Epoch 00005: val_loss improved from 3.09380 to 2.95170, saving model to model.h5
Epoch 6/60
 - 21s - loss: 2.8305 - val_loss: 2.8672

Epoch 00006: val_loss improved from 2.95170 to 2.86720, saving model to model.h5
Epoch 7/60
 - 21s - loss: 2.7590 - val_loss: 2.8011

Epoch 00007: val_loss improved from 2.86720 to 2.80111, saving model to model.h5
Epoch 8/60
 - 21s - l

In [158]:
dataset = load_clean_sentences('khasi-english.pkl')
train = load_clean_sentences('khasi-english-train.pkl')
test = load_clean_sentences('khasi-english-test.pkl')
# prepare english tokenizer
kha_tokenizer = create_tokenizer(dataset[:, 0])
kha_vocab_size = len(kha_tokenizer.word_index) + 1
kha_length = max_length(dataset[:, 0])
# prepare german tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 1])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 1])
# prepare data
trainX = encode_sequences(eng_tokenizer, eng_length, train[:, 1])
testX = encode_sequences(eng_tokenizer, eng_length, test[:, 1])

In [162]:
# load model
model = load_model('model.h5')

In [163]:
from numpy import argmax
def load_clean_sentences(filename):
    return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

# max sentence length
def max_length(lines):
    return max(len(line.split()) for line in lines)

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
    # integer encode sequences
    X = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    X = pad_sequences(X, maxlen=length, padding='post')
    return X

# map an integer to a word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None


def predict_sequence(model, tokenizer, source):
    prediction = model.predict(source, verbose=0)[0]
    integers = [argmax(vector) for vector in prediction]
    target = list()
    for i in integers:
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)

def evaluate_model(model, tokenizer, sources, raw_dataset):
    actual, predicted = list(), list()
    for i, source in enumerate(sources):
        # translate encoded source text
        source = source.reshape((1, source.shape[0]))
        translation = predict_sequence(model, eng_tokenizer, source)
        raw_target, raw_src = raw_dataset[i]
        if i < 10:
            print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
        actual.append([raw_target.split()])
        predicted.append(translation.split())
    # calculate BLEU score
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))


In [164]:
evaluate_model(model, eng_tokenizer, testX, test)

src=[he is lying on the baby], target=[u ban ia i khyllung], predicted=[i do i you i i i]
src=[she is a very beautiful woman], target=[ka long ka briew kaba bhabriew bha], predicted=[you what you you]
src=[tom is going to hurt you], target=[u tom un pynmynsaw ia pha], predicted=[i do i a i i i]
src=[the hill is always green], target=[u lum u jyrngam beit], predicted=[you do you you you want]
src=[you may do what you wish], target=[phi lah ban leh katba phi mon], predicted=[is what the the the he he]
src=[help], target=[iarap], predicted=[didnt]
src=[do you know what this is all about], target=[phi tip dei shaphang kaei kine baroh], predicted=[i do the i i i]
src=[poor cat], target=[bapli i miaw], predicted=[you to i]
src=[where is the toilet], target=[hangno ka painkhana], predicted=[you to you]
src=[tom behind you], target=[tom shadien jong phi], predicted=[you said of]


NameError: name 'corpus_bleu' is not defined